In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.options.display.max_rows = 1000

plt.style.use('seaborn-dark-palette')

In [2]:
df = pd.read_excel('Nz analysis.xlsx')

In [67]:
df.head()

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,bowler,runs_off_bat,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed,Player_out,Bowler_wk,is_dot,is_one,is_two,is_three,is_four,is_six,Phases,Total_runs
0,1187665,2019/20,2019-11-01,Hagley Oval,1,0.1,New Zealand,England,MJ Guptill,C Munro,SM Curran,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0,0,1,0,0,0,0,0,Powerplay,0.0
1,1187665,2019/20,2019-11-01,Hagley Oval,1,0.2,New Zealand,England,MJ Guptill,C Munro,SM Curran,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0,0,1,0,0,0,0,0,Powerplay,0.0
2,1187665,2019/20,2019-11-01,Hagley Oval,1,0.3,New Zealand,England,MJ Guptill,C Munro,SM Curran,0,1,0.0,0.0,0.0,1.0,0.0,0,0,0.0,0.0,0,0,1,0,0,0,0,0,Powerplay,0.0
3,1187665,2019/20,2019-11-01,Hagley Oval,1,0.4,New Zealand,England,C Munro,MJ Guptill,SM Curran,1,0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0,0,0,1,0,0,0,0,Powerplay,1.0
4,1187665,2019/20,2019-11-01,Hagley Oval,1,0.5,New Zealand,England,MJ Guptill,C Munro,SM Curran,1,0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0,0,0,1,0,0,0,0,Powerplay,1.0


In [4]:
def player_out(player_dismissed):
    if type(player_dismissed) == str:
        return 1
    else:
        return 0

In [5]:
df['Player_out'] = df.apply(lambda x: player_out(x['player_dismissed']), axis = 1)

In [6]:
def is_bowler_wk(wicket_type, player_dismissed):
    if type(player_dismissed) == str:
        if wicket_type not in ('run out','hit wicket','retired hurt',0):
            return 1
        else:
            return 0
    
    else:
        return 0

In [7]:
df['Bowler_wk'] = df.apply(lambda x: is_bowler_wk(x['wicket_type'], x['player_dismissed']), axis = 1)

In [8]:
df['is_dot'] = df.runs_off_bat.apply(lambda x: 1 if x == 0 else 0)
df['is_one'] = df.runs_off_bat.apply(lambda x: 1 if x == 1 else 0)
df['is_two'] = df.runs_off_bat.apply(lambda x: 1 if x == 2 else 0)
df['is_three'] = df.runs_off_bat.apply(lambda x: 1 if x == 3 else 0)
df['is_four'] = df.runs_off_bat.apply(lambda x: 1 if x == 4 else 0)
df['is_six'] = df.runs_off_bat.apply(lambda x: 1 if x == 6 else 0)

In [9]:
def Phase(ball):
  if ball <= 5.6:
    return 'Powerplay'
  elif ball <= 15.6:
    return 'Middle'
  else:
    return 'Death' 

In [10]:
df['Phases'] = df.apply(lambda x: Phase(x['ball']), axis = 1)

In [11]:
df = df.fillna(0)

In [12]:
def Economy_rate(runs_conceded, balls):
  return 6*runs_conceded/balls

def Boundary_percentage(balls, fours, six):
  return 100*(fours + six)/balls

def Dot_percentage(dots, balls):
  return 100*dots/balls

In [13]:
def Balls_per_boundary(balls, boundaries):
  if boundaries < 1:
    return balls/1
  else:
    return balls/boundaries

In [14]:
def Bowling_strike_rate(runs_conceded, wicket):
  if wicket <1:
    return runs_conceded/1 
  else:
    return runs_conceded/wicket

In [15]:

def Bowling_average(balls, wicket):
  if wicket < 1:
    return balls/1
  else:
    return balls/wicket

In [16]:
def Total_runs(runs_off_bat, wides, noballs):
  return runs_off_bat + wides + noballs

In [17]:
def Balls_per_extra(balls, extras):
  if extras < 1:
    return balls/1
  else:
    return balls/extras

def balls_per_wicket(balls, wicket):
  if wicket <1 :
    return balls/1
  else:
    return balls/wicket

In [18]:
df['Total_runs'] = df.apply(lambda x: Total_runs(x['runs_off_bat'], x['wides'],x['noballs']), axis = 1)

In [19]:
df.head(1)

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,bowler,runs_off_bat,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed,Player_out,Bowler_wk,is_dot,is_one,is_two,is_three,is_four,is_six,Phases,Total_runs
0,1187665,2019/20,2019-11-01,Hagley Oval,1,0.1,New Zealand,England,MJ Guptill,C Munro,SM Curran,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0,0,1,0,0,0,0,0,Powerplay,0.0


In [20]:
def Bowler_stats(df, phase, bowling_team):
    df = df[df.Phases == phase]
    df = df[df.bowling_team == bowling_team]

    runs  = df.groupby(['bowler'])['Total_runs'].sum().reset_index().rename(columns = {"Total_runs": "runs_conceded"})
    balls = df.groupby(['bowler'])['ball'].count().reset_index().rename(columns = {'ball':"balls"})
    dismissals = df.groupby(['bowler'])['Bowler_wk'].sum().reset_index().rename(columns = {'Bowler_wk':'wicket'})
    innings = df.groupby(['bowler'])['match_id'].apply(lambda x: len(list(np.unique(x)))).reset_index().rename(columns = {"match_id": "innings"})
    
    ones = df.groupby(['bowler'])['is_one'].sum().reset_index().rename(columns = {'is_one':'ones'})
    twos = df.groupby(['bowler'])['is_two'].sum().reset_index().rename(columns = {'is_two':'twos'})
    threes = df.groupby(['bowler'])['is_three'].sum().reset_index().rename(columns = {'is_three':'threes'})
    fours = df.groupby(['bowler'])['is_four'].sum().reset_index().rename(columns = {'is_four':'fours'})
    sixes = df.groupby(['bowler'])['is_six'].sum().reset_index().rename(columns = {'is_six': 'six'})
    dots = df.groupby(['bowler'])['is_dot'].sum().reset_index().rename(columns = {"is_dot":"dots"})
    extras = df.groupby(['bowler'])['wides', 'noballs'].sum().reset_index()

    df = pd.merge(innings,runs, on = ['bowler']).merge(balls, on = ['bowler']).merge(dismissals, on = ['bowler']).merge(dots, on = ['bowler']).merge(ones, on = ['bowler']).merge(twos, on = ['bowler']).merge(threes, on = ['bowler']).merge(fours, on = ['bowler']).merge(sixes, on = ['bowler']).merge(extras, on = ['bowler'])

    df['Economy_rate'] = df.apply(lambda x: Economy_rate(x['runs_conceded'] , x['balls']), axis = 1)
    df['Bowling_SR'] = df.apply(lambda x: Bowling_strike_rate(x['runs_conceded'], x['wicket']), axis = 1 )
    df['Bowling_AVG'] = df.apply(lambda x: Bowling_average(x['balls'], x['wicket']), axis = 1)
    df['Boundary_percentage'] = df.apply(lambda x: Boundary_percentage(x['balls'], x['fours'], x['six']), axis = 1)
    df['Dot_percentage'] = df.apply(lambda x: Dot_percentage(x['dots'], x['balls']), axis = 1)
    df['BPB'] = df.apply(lambda x: Balls_per_boundary(x['balls'], x['fours'] + x['six']), axis = 1)
    df['Balls_per_extra'] = df.apply(lambda x: Balls_per_extra(x['balls'], x['wides'] + x['noballs']), axis = 1)
    return df
    

In [21]:
Powerplay_bowling = Bowler_stats(df, 'Powerplay', 'New Zealand').sort_values(by = 'wicket', ascending = False).set_index('bowler')


In [22]:
Middle_bowling = Bowler_stats(df, 'Middle', 'New Zealand').sort_values(by = 'wicket', ascending = False).set_index('bowler')

In [66]:
Death_bowling = Bowler_stats(df, 'Death', 'New Zealand').sort_values(by = 'wicket', ascending = False).set_index('bowler')

#### NZ's Current Bowling attack

In [24]:
Current_PP = Powerplay_bowling.loc[['TD Astle','TA Boult','LH Ferguson','KA Jamieson','JDS Neesham','MJ Santner','IS Sodhi','TG Southee']]


In [25]:
Current_middle = Middle_bowling.loc[['TD Astle','TA Boult','LH Ferguson','KA Jamieson','JDS Neesham','MJ Santner','IS Sodhi','TG Southee']]

In [26]:
Current_death = Death_bowling.loc[['TA Boult','LH Ferguson','KA Jamieson','JDS Neesham','MJ Santner','IS Sodhi','TG Southee']]

In [27]:
Current_PP['Bowling_Avg'] = Current_PP.apply(lambda x: balls_per_wicket(x['balls'], x['wicket']), axis = 1 )
Current_PP = Current_PP[['innings','runs_conceded','balls','wicket','Economy_rate','Bowling_SR','Dot_percentage','BPB','Bowling_Avg']]
Current_PP

,innings,runs_conceded,balls,wicket,Economy_rate,Bowling_SR,Dot_percentage,BPB,Bowling_Avg
bowler,,,,,,,,,
TD Astle,1,13.0,12,4,6.500000,3.250000,66.666667,6.000000,3.000000
TA Boult,17,268.0,202,8,7.960396,33.500000,52.970297,4.926829,25.250000
LH Ferguson,12,136.0,115,9,7.095652,15.111111,53.043478,5.476190,12.777778
KA Jamieson,8,95.0,65,1,8.769231,95.000000,52.307692,4.642857,65.000000
JDS Neesham,2,18.0,12,0,9.000000,18.000000,41.666667,4.000000,12.000000
MJ Santner,20,179.0,144,7,7.458333,25.571429,52.777778,5.760000,20.571429
IS Sodhi,5,60.0,36,2,10.000000,30.000000,41.666667,4.000000,18.000000
TG Southee,41,630.0,526,25,7.186312,25.200000,49.619772,5.844444,21.040000


In [28]:
Current_middle['Bowling_Avg'] = Current_middle.apply(lambda x: balls_per_wicket(x['balls'], x['wicket']), axis = 1 )
Current_death['Bowling_Avg'] = Current_death.apply(lambda x: balls_per_wicket(x['balls'], x['wicket']), axis = 1 )


In [29]:
Current_PP

,innings,runs_conceded,balls,wicket,Economy_rate,Bowling_SR,Dot_percentage,BPB,Bowling_Avg
bowler,,,,,,,,,
TD Astle,1,13.0,12,4,6.500000,3.250000,66.666667,6.000000,3.000000
TA Boult,17,268.0,202,8,7.960396,33.500000,52.970297,4.926829,25.250000
LH Ferguson,12,136.0,115,9,7.095652,15.111111,53.043478,5.476190,12.777778
KA Jamieson,8,95.0,65,1,8.769231,95.000000,52.307692,4.642857,65.000000
JDS Neesham,2,18.0,12,0,9.000000,18.000000,41.666667,4.000000,12.000000
MJ Santner,20,179.0,144,7,7.458333,25.571429,52.777778,5.760000,20.571429
IS Sodhi,5,60.0,36,2,10.000000,30.000000,41.666667,4.000000,18.000000
TG Southee,41,630.0,526,25,7.186312,25.200000,49.619772,5.844444,21.040000


In [30]:
Current_middle = Current_middle[['innings','runs_conceded','balls','wicket','Economy_rate','Bowling_SR','Dot_percentage','BPB','Bowling_Avg']]
Current_middle

,innings,runs_conceded,balls,wicket,Economy_rate,Bowling_SR,Dot_percentage,BPB,Bowling_Avg
bowler,,,,,,,,,
TD Astle,1,28.0,24,3,7.000000,9.333333,41.666667,8.000000,8.000000
TA Boult,12,118.0,95,6,7.452632,19.666667,41.052632,6.333333,15.833333
LH Ferguson,11,113.0,107,7,6.336449,16.142857,48.598131,8.230769,15.285714
KA Jamieson,8,107.0,83,1,7.734940,107.000000,39.759036,6.384615,83.000000
JDS Neesham,9,157.0,87,4,10.827586,39.250000,25.287356,4.578947,21.750000
MJ Santner,33,675.0,555,29,7.297297,23.275862,34.594595,9.098361,19.137931
IS Sodhi,40,980.0,764,37,7.696335,26.486486,34.947644,7.564356,20.648649
TG Southee,28,239.0,170,10,8.435294,23.900000,37.058824,5.666667,17.000000


In [31]:

Current_death = Current_death[['innings','runs_conceded','balls','wicket','Economy_rate','Bowling_SR','Dot_percentage','BPB','Bowling_Avg']]
Current_death

,innings,runs_conceded,balls,wicket,Economy_rate,Bowling_SR,Dot_percentage,BPB,Bowling_Avg
bowler,,,,,,,,,
TA Boult,13,182.0,105,6,10.400000,30.333333,26.666667,4.038462,17.500000
LH Ferguson,7,35.0,37,5,5.675676,7.000000,45.945946,9.250000,7.400000
KA Jamieson,6,79.0,34,2,13.941176,39.500000,23.529412,2.833333,17.000000
JDS Neesham,4,59.0,21,3,16.857143,19.666667,28.571429,1.909091,7.000000
MJ Santner,4,29.0,17,3,10.235294,9.666667,35.294118,5.666667,5.666667
IS Sodhi,15,169.0,88,8,11.522727,21.125000,27.272727,4.000000,11.000000
TG Southee,33,390.0,261,17,8.965517,22.941176,33.716475,5.931818,15.352941


In [32]:
def Bowler_stats_1(df, bowling_team):
    df = df[df.bowling_team == bowling_team]

    runs  = df.groupby(['bowler'])['Total_runs'].sum().reset_index().rename(columns = {"Total_runs": "runs_conceded"})
    balls = df.groupby(['bowler'])['ball'].count().reset_index().rename(columns = {'ball':"balls"})
    dismissals = df.groupby(['bowler'])['Bowler_wk'].sum().reset_index().rename(columns = {'Bowler_wk':'wicket'})
    innings = df.groupby(['bowler'])['match_id'].apply(lambda x: len(list(np.unique(x)))).reset_index().rename(columns = {"match_id": "innings"})
    
    ones = df.groupby(['bowler'])['is_one'].sum().reset_index().rename(columns = {'is_one':'ones'})
    twos = df.groupby(['bowler'])['is_two'].sum().reset_index().rename(columns = {'is_two':'twos'})
    threes = df.groupby(['bowler'])['is_three'].sum().reset_index().rename(columns = {'is_three':'threes'})
    fours = df.groupby(['bowler'])['is_four'].sum().reset_index().rename(columns = {'is_four':'fours'})
    sixes = df.groupby(['bowler'])['is_six'].sum().reset_index().rename(columns = {'is_six': 'six'})
    dots = df.groupby(['bowler'])['is_dot'].sum().reset_index().rename(columns = {"is_dot":"dots"})
    extras = df.groupby(['bowler'])['wides', 'noballs'].sum().reset_index()

    df = pd.merge(innings,runs, on = ['bowler']).merge(balls, on = ['bowler']).merge(dismissals, on = ['bowler']).merge(dots, on = ['bowler']).merge(ones, on = ['bowler']).merge(twos, on = ['bowler']).merge(threes, on = ['bowler']).merge(fours, on = ['bowler']).merge(sixes, on = ['bowler']).merge(extras, on = ['bowler'])

    df['Economy_rate'] = df.apply(lambda x: Economy_rate(x['runs_conceded'] , x['balls']), axis = 1)
    df['Bowling_SR'] = df.apply(lambda x: Bowling_strike_rate(x['runs_conceded'], x['wicket']), axis = 1 )
    df['Bowling_AVG'] = df.apply(lambda x: Bowling_average(x['balls'], x['wicket']), axis = 1)
    df['Boundary_percentage'] = df.apply(lambda x: Boundary_percentage(x['balls'], x['fours'], x['six']), axis = 1)
    df['Dot_percentage'] = df.apply(lambda x: Dot_percentage(x['dots'], x['balls']), axis = 1)
    df['BPB'] = df.apply(lambda x: Balls_per_boundary(x['balls'], x['fours'] + x['six']), axis = 1)
    df['Balls_per_extra'] = df.apply(lambda x: Balls_per_extra(x['balls'], x['wides'] + x['noballs']), axis = 1)

    return df

In [33]:
Bowler_stats_1(df, 'New Zealand').set_index('bowler').loc[['TA Boult','LH Ferguson','KA Jamieson','JDS Neesham','MJ Santner','IS Sodhi','TG Southee']]

,innings,runs_conceded,balls,wicket,dots,ones,twos,threes,fours,six,wides,noballs,Economy_rate,Bowling_SR,Bowling_AVG,Boundary_percentage,Dot_percentage,BPB,Balls_per_extra
bowler,,,,,,,,,,,,,,,,,,,
TA Boult,17,568.0,402,20,174,116,27,3,63,19,21.0,2.0,8.477612,28.400000,20.100000,20.398010,43.283582,4.902439,17.478261
LH Ferguson,12,284.0,259,21,130,76,14,1,32,6,13.0,0.0,6.579151,13.523810,12.333333,14.671815,50.193050,6.815789,19.923077
KA Jamieson,8,281.0,182,4,75,57,9,2,22,17,9.0,1.0,9.263736,70.250000,45.500000,21.428571,41.208791,4.666667,18.200000
JDS Neesham,10,234.0,120,7,33,41,13,0,17,16,2.0,1.0,11.700000,33.428571,17.142857,27.500000,27.500000,3.636364,40.000000
MJ Santner,34,883.0,716,39,274,300,49,4,48,41,32.0,3.0,7.399441,22.641026,18.358974,12.430168,38.268156,8.044944,20.457143
IS Sodhi,40,1209.0,888,47,306,388,60,2,67,65,33.0,4.0,8.168919,25.723404,18.893617,14.864865,34.459459,6.727273,24.000000
TG Southee,41,1259.0,957,52,412,307,67,7,114,50,40.0,1.0,7.893417,24.211538,18.403846,17.136886,43.051202,5.835366,23.341463


### **NZ Batting analysis**

In [34]:
df.head()

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,bowler,runs_off_bat,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed,Player_out,Bowler_wk,is_dot,is_one,is_two,is_three,is_four,is_six,Phases,Total_runs
0,1187665,2019/20,2019-11-01,Hagley Oval,1,0.1,New Zealand,England,MJ Guptill,C Munro,SM Curran,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0,0,1,0,0,0,0,0,Powerplay,0.0
1,1187665,2019/20,2019-11-01,Hagley Oval,1,0.2,New Zealand,England,MJ Guptill,C Munro,SM Curran,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0,0,1,0,0,0,0,0,Powerplay,0.0
2,1187665,2019/20,2019-11-01,Hagley Oval,1,0.3,New Zealand,England,MJ Guptill,C Munro,SM Curran,0,1,0.0,0.0,0.0,1.0,0.0,0,0,0.0,0.0,0,0,1,0,0,0,0,0,Powerplay,0.0
3,1187665,2019/20,2019-11-01,Hagley Oval,1,0.4,New Zealand,England,C Munro,MJ Guptill,SM Curran,1,0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0,0,0,1,0,0,0,0,Powerplay,1.0
4,1187665,2019/20,2019-11-01,Hagley Oval,1,0.5,New Zealand,England,MJ Guptill,C Munro,SM Curran,1,0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0,0,0,1,0,0,0,0,Powerplay,1.0


In [35]:
df1 = df.copy()

In [36]:
df1.head()

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,bowler,runs_off_bat,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed,Player_out,Bowler_wk,is_dot,is_one,is_two,is_three,is_four,is_six,Phases,Total_runs
0,1187665,2019/20,2019-11-01,Hagley Oval,1,0.1,New Zealand,England,MJ Guptill,C Munro,SM Curran,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0,0,1,0,0,0,0,0,Powerplay,0.0
1,1187665,2019/20,2019-11-01,Hagley Oval,1,0.2,New Zealand,England,MJ Guptill,C Munro,SM Curran,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0,0,1,0,0,0,0,0,Powerplay,0.0
2,1187665,2019/20,2019-11-01,Hagley Oval,1,0.3,New Zealand,England,MJ Guptill,C Munro,SM Curran,0,1,0.0,0.0,0.0,1.0,0.0,0,0,0.0,0.0,0,0,1,0,0,0,0,0,Powerplay,0.0
3,1187665,2019/20,2019-11-01,Hagley Oval,1,0.4,New Zealand,England,C Munro,MJ Guptill,SM Curran,1,0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0,0,0,1,0,0,0,0,Powerplay,1.0
4,1187665,2019/20,2019-11-01,Hagley Oval,1,0.5,New Zealand,England,MJ Guptill,C Munro,SM Curran,1,0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0,0,0,1,0,0,0,0,Powerplay,1.0


In [37]:
def Phases_batting(ball):
  if ball <= 5.6:
    return 'Powerplay'
  elif ball <= 15.6:
    return 'Middle'
  elif ball <= 17.6:
    return 'Death(17th and 18th over)'
  else:
    return 'Death(19th and 20th over)'


In [38]:
df1['Phases_batting'] = df.apply(lambda x: Phases_batting(x['ball']), axis = 1)


In [39]:


def Dot_percentage(dots, balls):
  return 100*dots/balls
  
def Balls_per_boundary(balls, boundaries):
  if boundaries < 1:
    return balls/1
  else:
    return balls/boundaries

def Balls_per_innings(balls, innings):
  return balls/innings

def Runs_per_innings(runs_conceded, innings):
  return runs_conceded/innings

def Strike_rate(runs_conceded, balls):
  return 100*runs_conceded/balls

def Strike_rotation_percent(balls, ones, threes):
  return 100*(ones + threes)/balls

def four_six_ratio(fours, six):
  if fours< 1:
    return six/1
  else:
    return six/fours

In [40]:
def Batsman_stats(df, batting_team, phase):
    df = df[df.batting_team == batting_team]
    df = df[df.Phases_batting == phase]

    runs  = df.groupby(['striker'])['runs_off_bat'].sum().reset_index().rename(columns = {"runs_off_bat": "runs_conceded"})
    balls = df.groupby(['striker'])['ball'].count().reset_index().rename(columns = {'ball':"balls"})
    dismissals = df.groupby(['striker'])['Bowler_wk'].sum().reset_index().rename(columns = {'Bowler_wk':'wicket'})
    innings = df.groupby(['striker'])['match_id'].apply(lambda x: len(list(np.unique(x)))).reset_index().rename(columns = {"match_id": "innings"})
    
    ones = df.groupby(['striker'])['is_one'].sum().reset_index().rename(columns = {'is_one':'ones'})
    twos = df.groupby(['striker'])['is_two'].sum().reset_index().rename(columns = {'is_two':'twos'})
    threes = df.groupby(['striker'])['is_three'].sum().reset_index().rename(columns = {'is_three':'threes'})
    fours = df.groupby(['striker'])['is_four'].sum().reset_index().rename(columns = {'is_four':'fours'})
    sixes = df.groupby(['striker'])['is_six'].sum().reset_index().rename(columns = {'is_six': 'six'})
    dots = df.groupby(['striker'])['is_dot'].sum().reset_index().rename(columns = {"is_dot":"dots"})
    extras = df.groupby(['striker'])['wides', 'noballs'].sum().reset_index()

    df = pd.merge(innings,runs, on = ['striker']).merge(balls, on = ['striker']).merge(dismissals, on = ['striker']).merge(dots, on = ['striker']).merge(ones, on = ['striker']).merge(twos, on = ['striker']).merge(threes, on = ['striker']).merge(fours, on = ['striker']).merge(sixes, on = ['striker']).merge(extras, on = ['striker'])
  
    df['Strike_rate'] = df.apply(lambda x: Strike_rate(x['runs_conceded'], x['balls']), axis = 1)
    df['RPI'] = df.apply(lambda x: Runs_per_innings(x['runs_conceded'], x['innings']), axis = 1)
    df['BPI'] = df.apply(lambda x: Balls_per_innings(x['balls'], x['innings']), axis = 1)
    df['BPB'] = df.apply(lambda x: Balls_per_boundary(x['balls'], x['fours'] + x['six']), axis = 1)
    df['Dot_percentage'] = df.apply(lambda x:Dot_percentage(x['dots'], x['balls']), axis = 1)
    df['Strike_rotation_percent'] = df.apply(lambda x: Strike_rotation_percent(x['balls'], x['ones'], x['threes']), axis = 1)
    df['sixes:fours'] = df.apply(lambda x: four_six_ratio(x['fours'], x['six']), axis = 1)
    return df

In [41]:
Batting_PP = Batsman_stats(df1, 'New Zealand','Powerplay').set_index('striker').sort_values(by = 'runs_conceded', ascending = False)


In [42]:
Batting_middle = Batsman_stats(df1, 'New Zealand','Middle').set_index('striker').sort_values(by = 'runs_conceded', ascending = False)


In [65]:
Batting_death_17and18 = Batsman_stats(df1, 'New Zealand','Death(17th and 18th over)').set_index('striker').sort_values(by = 'runs_conceded', ascending = False)


In [44]:
Batting_death_19and20 = Batsman_stats(df1, 'New Zealand','Death(19th and 20th over)').set_index('striker').sort_values(by = 'runs_conceded', ascending = False)


**NZ's Current Batting Analysis**

In [45]:
Batting_PP = Batting_PP.loc[['KS Williamson','MJ Guptill','MJ Santner','DP Conway','TL Seifert','GD Phillips','DJ Mitchell']]
Batting_PP = Batting_PP[['innings','runs_conceded','balls','wicket','Strike_rate','RPI','BPI','BPB','Dot_percentage','Strike_rotation_percent','sixes:fours']].round(2)

In [46]:
Batting_middle = Batting_middle.loc[['KS Williamson','MJ Guptill','MJ Santner','DP Conway','TL Seifert','GD Phillips','DJ Mitchell','TG Southee','IS Sodhi','JDS Neesham']]
Batting_middle = Batting_middle[['innings','runs_conceded','balls','wicket','Strike_rate','RPI','BPI','BPB','Dot_percentage','Strike_rotation_percent','sixes:fours']].round(2)

In [47]:
Batting_death_17and18 = Batting_death_17and18.loc[['KS Williamson','MJ Guptill','MJ Santner','DP Conway','TL Seifert','GD Phillips','DJ Mitchell','TG Southee','IS Sodhi','JDS Neesham']]
Batting_death_17and18 = Batting_death_17and18[['innings','runs_conceded','balls','wicket','Strike_rate','RPI','BPI','BPB','Dot_percentage','Strike_rotation_percent','sixes:fours']].round(2)

In [48]:
Batting_death_19and20 = Batting_death_19and20.loc[['KS Williamson','MJ Santner','DP Conway','TL Seifert','GD Phillips','DJ Mitchell','TG Southee','IS Sodhi','JDS Neesham']]
Batting_death_19and20 = Batting_death_19and20[['innings','runs_conceded','balls','wicket','Strike_rate','RPI','BPI','BPB','Dot_percentage','Strike_rotation_percent','sixes:fours']].round(2)

In [49]:
Batting_PP

,innings,runs_conceded,balls,wicket,Strike_rate,RPI,BPI,BPB,Dot_percentage,Strike_rotation_percent,sixes:fours
striker,,,,,,,,,,,
KS Williamson,16,108,119,5,90.76,6.75,7.44,8.50,50.42,35.29,0.17
MJ Guptill,36,721,527,22,136.81,20.03,14.64,4.83,45.54,29.60,0.58
MJ Santner,2,8,12,0,66.67,4.00,6.00,12.00,66.67,16.67,0.00
DP Conway,9,98,74,1,132.43,10.89,8.22,4.93,39.19,37.84,0.15
TL Seifert,23,294,257,12,114.40,12.78,11.17,5.98,50.58,26.46,0.30
GD Phillips,9,68,80,4,85.00,7.56,8.89,10.00,52.50,37.50,0.60
DJ Mitchell,2,2,4,1,50.00,1.00,2.00,4.00,50.00,50.00,0.00


In [50]:
Batting_middle

,innings,runs_conceded,balls,wicket,Strike_rate,RPI,BPI,BPB,Dot_percentage,Strike_rotation_percent,sixes:fours
striker,,,,,,,,,,,
KS Williamson,18,464,327,12,141.90,25.78,18.17,6.41,27.83,48.93,0.82
MJ Guptill,14,353,213,12,165.73,25.21,15.21,5.20,24.88,47.42,1.73
MJ Santner,9,112,94,6,119.15,12.44,10.44,9.40,35.11,40.43,0.43
DP Conway,9,242,190,5,127.37,26.89,21.11,7.04,27.37,50.53,0.17
TL Seifert,17,355,227,14,156.39,20.88,13.35,5.04,29.96,44.49,1.05
GD Phillips,14,354,205,9,172.68,25.29,14.64,4.36,28.78,39.02,0.88
DJ Mitchell,10,67,64,4,104.69,6.70,6.40,16.00,34.38,48.44,1.00
TG Southee,5,83,54,4,153.70,16.60,10.80,4.91,44.44,27.78,2.67
IS Sodhi,5,20,26,0,76.92,4.00,5.20,26.00,46.15,46.15,1.00


In [51]:
Batting_death_17and18

,innings,runs_conceded,balls,wicket,Strike_rate,RPI,BPI,BPB,Dot_percentage,Strike_rotation_percent,sixes:fours
striker,,,,,,,,,,,
KS Williamson,6,44,33,2,133.33,7.33,5.50,5.50,42.42,18.18,0.00
MJ Guptill,2,15,7,2,214.29,7.50,3.50,3.50,42.86,14.29,2.00
MJ Santner,11,36,36,4,100.00,3.27,3.27,18.00,36.11,44.44,1.00
DP Conway,5,61,25,0,244.00,12.20,5.00,2.50,16.00,36.00,0.67
TL Seifert,5,21,27,1,77.78,4.20,5.40,9.00,55.56,33.33,0.00
GD Phillips,4,41,22,1,186.36,10.25,5.50,3.14,50.00,4.55,0.75
DJ Mitchell,4,54,23,0,234.78,13.50,5.75,2.30,8.70,43.48,0.11
TG Southee,2,7,5,0,140.00,3.50,2.50,5.00,40.00,20.00,0.00
IS Sodhi,6,23,18,2,127.78,3.83,3.00,6.00,33.33,50.00,0.50


In [52]:
Batting_death_19and20

,innings,runs_conceded,balls,wicket,Strike_rate,RPI,BPI,BPB,Dot_percentage,Strike_rotation_percent,sixes:fours
striker,,,,,,,,,,,
KS Williamson,5,27,16,4,168.75,5.40,3.20,3.20,37.50,31.25,0.25
MJ Santner,14,100,52,2,192.31,7.14,3.71,3.71,21.15,38.46,0.56
DP Conway,5,72,31,2,232.26,14.40,6.20,2.82,19.35,25.81,0.57
TL Seifert,7,39,28,2,139.29,5.57,4.00,5.60,50.00,25.00,4.00
GD Phillips,3,27,17,1,158.82,9.00,5.67,4.25,35.29,29.41,0.33
DJ Mitchell,5,25,21,2,119.05,5.00,4.20,7.00,42.86,33.33,0.50
TG Southee,11,56,39,3,143.59,5.09,3.55,6.50,33.33,30.77,0.50
IS Sodhi,4,22,17,3,129.41,5.50,4.25,5.67,58.82,23.53,3.00
JDS Neesham,3,35,21,2,166.67,11.67,7.00,4.20,47.62,23.81,4.00


##**Leaders data**

### **Middle overs**

In [53]:
Leader_data_middle = pd.read_csv('leadersdata.csv')

In [54]:
Leader_data_middle['BPB'] = ((Leader_data_middle['B'])/(Leader_data_middle['4s'] + Leader_data_middle['6s'])).round(2)
Leader_data_middle['RPI'] = (Leader_data_middle['R']/Leader_data_middle['I']).round(2)
Leader_data_middle['BPI'] = (Leader_data_middle['B']/Leader_data_middle['I']).round(2)

# This data will be used as referecne for comapring players in the middle overs of NZ

### **Powerplay**

In [55]:
leader_data_pp = pd.read_csv('leadersdata_pp.csv')

In [56]:
leader_data_pp['BPB'] = ((leader_data_pp['B'])/(leader_data_pp['4s'] + leader_data_pp['6s'])).round(2)
leader_data_pp['RPI'] = (leader_data_pp['R']/leader_data_pp['I']).round(2)
leader_data_pp['BPI'] = (leader_data_pp['B']/leader_data_pp['I']).round(2)

In [64]:
leader_data_pp.sort_values(by = 'R', ascending = False).head()

,Player,I,R,B,Outs,Avg,SR,HS,4s,6s,50,100,BPB,RPI,BPI
467,PR Stirling,42,874,621,18,48.56,140.74,67,111,32,1,0,4.34,20.81,14.79
369,MJ Guptill,35,726,509,22,33.00,142.63,50,67,42,1,0,4.67,20.74,14.54
323,KJ O'Brien,37,666,453,23,28.96,147.02,39,75,31,0,0,4.27,18.00,12.24
97,Babar Azam,40,665,516,14,47.50,128.88,45,98,3,0,0,5.11,16.62,12.90
499,RG Sharma,38,664,491,21,31.62,135.23,50,73,32,1,0,4.68,17.47,12.92


### **Overall batsman stats**

In [58]:
def Batsman_stats_1(df, batting_team):
    df = df[df.batting_team == batting_team]

    runs  = df.groupby(['striker'])['runs_off_bat'].sum().reset_index().rename(columns = {"runs_off_bat": "runs_conceded"})
    balls = df.groupby(['striker'])['ball'].count().reset_index().rename(columns = {'ball':"balls"})
    dismissals = df.groupby(['striker'])['Bowler_wk'].sum().reset_index().rename(columns = {'Bowler_wk':'wicket'})
    innings = df.groupby(['striker'])['match_id'].apply(lambda x: len(list(np.unique(x)))).reset_index().rename(columns = {"match_id": "innings"})
    
    ones = df.groupby(['striker'])['is_one'].sum().reset_index().rename(columns = {'is_one':'ones'})
    twos = df.groupby(['striker'])['is_two'].sum().reset_index().rename(columns = {'is_two':'twos'})
    threes = df.groupby(['striker'])['is_three'].sum().reset_index().rename(columns = {'is_three':'threes'})
    fours = df.groupby(['striker'])['is_four'].sum().reset_index().rename(columns = {'is_four':'fours'})
    sixes = df.groupby(['striker'])['is_six'].sum().reset_index().rename(columns = {'is_six': 'six'})
    dots = df.groupby(['striker'])['is_dot'].sum().reset_index().rename(columns = {"is_dot":"dots"})
    extras = df.groupby(['striker'])['wides', 'noballs'].sum().reset_index()

    df = pd.merge(innings,runs, on = ['striker']).merge(balls, on = ['striker']).merge(dismissals, on = ['striker']).merge(dots, on = ['striker']).merge(ones, on = ['striker']).merge(twos, on = ['striker']).merge(threes, on = ['striker']).merge(fours, on = ['striker']).merge(sixes, on = ['striker']).merge(extras, on = ['striker'])
  
    df['Strike_rate'] = df.apply(lambda x: Strike_rate(x['runs_conceded'], x['balls']), axis = 1)
    df['RPI'] = df.apply(lambda x: Runs_per_innings(x['runs_conceded'], x['innings']), axis = 1)
    df['BPI'] = df.apply(lambda x: Balls_per_innings(x['balls'], x['innings']), axis = 1)
    df['BPB'] = df.apply(lambda x: Balls_per_boundary(x['balls'], x['fours'] + x['six']), axis = 1)
    df['Dot_percentage'] = df.apply(lambda x:Dot_percentage(x['dots'], x['balls']), axis = 1)
    return df

In [63]:
Batsman_stats_1(df1, 'New Zealand').sort_values(by = 'runs_conceded', ascending = False).set_index('striker').head()

,innings,runs_conceded,balls,wicket,dots,ones,twos,threes,fours,six,wides,noballs,Strike_rate,RPI,BPI,BPB,Dot_percentage
striker,,,,,,,,,,,,,,,,,
MJ Guptill,36,1089,747,36,296,257,40,1,84,68,27.0,2.0,145.783133,30.250000,20.750000,4.914474,39.625167
C Munro,30,1005,647,28,241,228,25,7,85,61,18.0,3.0,155.332303,33.500000,21.566667,4.431507,37.248841
TL Seifert,32,709,539,29,227,183,31,2,59,37,10.0,0.0,131.539889,22.156250,16.843750,5.614583,42.115028
LRPL Taylor,28,653,535,17,166,255,52,0,39,23,22.0,3.0,122.056075,23.321429,19.107143,8.629032,31.028037
KS Williamson,25,643,495,23,171,211,35,2,50,26,7.0,1.0,129.898990,25.720000,19.800000,6.513158,34.545455


In [60]:
## Code to give highest score of the respective player 

In [61]:
match_runs = df.groupby(['striker','match_id'])['runs_off_bat'].sum().to_frame().reset_index()

In [68]:
match_runs.groupby(['striker'])['runs_off_bat'].max().to_frame().sort_values(by = 'runs_off_bat', ascending = False).loc[['KS Williamson','MJ Guptill','MJ Santner','DP Conway','TL Seifert','GD Phillips','DJ Mitchell','TG Southee','IS Sodhi','JDS Neesham']]

,runs_off_bat
striker,
KS Williamson,106
MJ Guptill,105
MJ Santner,37
DP Conway,99
TL Seifert,84
GD Phillips,108
DJ Mitchell,34
TG Southee,39
IS Sodhi,16
